In [1]:
import requests 
import pandas as pd
from scipy import stats
import regex as re
import numpy as np

# Correlation between population of city and Win/loss ratio n NHL 2018

In [91]:

cities=pd.read_html("Datasets/wikipedia_data.html")[1]    #returns 1st table from wiki page
cities=cities.iloc[:-1,[0,3,5,6,7,8]]       #all rows except last for mentioned column numbers
cities
nhl_df = pd.read_csv("Datasets/nhl.csv")
nhl_df

def clean_nhl_df():
    cities=pd.read_html("Datasets/wikipedia_data.html")[1]    
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]       
    nhl_df = pd.read_csv("Datasets/nhl.csv")
    
    #Cleaning NHL DF
    nhl_df = nhl_df[nhl_df["year"] ==2018]    #only retunrs data with 2018 
    nhl_df["team"] = nhl_df["team"].str.replace("*", "")  #removes all the asterisks from team names
    nhl_df["team"]= nhl_df["team"].apply(lambda x: x.split(" ")[-1])   #splits team names and keeps last word of team 
    
    
    #clean NHL row column in cities df
    cities["NHL"] = cities["NHL"].str.replace(r"\[.+\]", "")
    cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                           "KingsDucks" : "Kings,Ducks",
                                           "Red Wings": "Red,Wings",
                                           "Maple Leafs": "Maple,Leafs",
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights"})  #splitting teams with spaces into words
                                              
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")    #splits any row in NBA with "," into seperate row 
    
    #merge both dfs based on team name 
    merged_df = pd.merge(cities, nhl_df, left_on= "NHL", right_on= "team")   #merging based on columns with team names
    merged_df= merged_df.iloc[:,[0,1,5,6,8,9]] #filtering only necessary columns
    merged_df["W-L%"] = merged_df["W"].astype("float") / (merged_df["W"].astype("float") + merged_df["L"].astype("float"))  #creating calculated column with W-L ratio 
    merged_df["Population (2016 est.)[8]"] =    merged_df["Population (2016 est.)[8]"].astype("float")    #converting population column to float 
    
    
    #Reassigning values for duplicate areas (i.e NY, LS)
    merged_df.loc[merged_df["Metropolitan area"] == "New York City", "W-L%"] = 0.5182013333333334   #assigns mean of 3 W-L% for NY
    merged_df.loc[merged_df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.6228945 #assigns mean of 3 W-L% for NY
    merged_df = merged_df.drop_duplicates(subset= "Metropolitan area")
    merged_df = merged_df.reset_index()   #creates another ordered index
    merged_df = merged_df.drop(columns="index")   #removes original filtered index column 
    return merged_df

def nhl_correlation():
    corr_df= clean_nhl_df()
    
    population_by_region = corr_df["Population (2016 est.)[8]"]   #creates column for comparison
    win_loss_by_region = corr_df["W-L%"]         #creates w-l column for comparison
    
    #correlation = stats.pearsonr(corr_df["Population (2016 est.)[8]"], corr_df["W-L%"])  #Direct method
    #correlation = correlation[0]   #only keeps corr value which is 0.012
    
    result = stats.pearsonr(population_by_region, win_loss_by_region)
    return result[0]

clean_nhl_df() 
nhl_correlation()

C:\Users\mendo\AppData\Local\Temp/ipykernel_15492/2550715962.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nhl_df["team"] = nhl_df["team"].str.replace("*", "")  #removes all the asterisks from team names
C:\Users\mendo\AppData\Local\Temp/ipykernel_15492/2550715962.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  cities["NHL"] = cities["NHL"].str.replace(r"\[.+\]", "")


0.012485959345532899

# Correlation between population of city and Win/loss ratio in NBA 2018

In [98]:

def clean_nba_df():
    nba_df=pd.read_csv("Datasets/nba.csv")
    cities=pd.read_html("Datasets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    #cleaning nba_df
    nba_df  = nba_df[nba_df["year"] == 2018]
    #nba_df["team"]= nba_df["team"].str.replace(r"\*.+", "")
    #nba_df["team"]= nba_df["team"].str.replace(r"\(.+\)", "")
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    nba_df["team"]= nba_df["team"].apply(lambda x: x.split(" ")[-1])
    #might need to another split line 
    
    #cleaning wikipedia_df
    cities["NBA"] = cities["NBA"].str.replace(r"\[.+\]", "")
    cities["NBA"] = cities["NBA"].replace({"KnicksNets": "Knicks,Nets",
                                        "LakersClippers": "Lakers,Clippers",
                                          "Trail Blazers": "Blazers"})
    
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))                               
    cities = cities.explode("NBA")   #splits any row in NBA with "," into seperate row 
    
    #merging both dfs
    merged_df = pd.merge(cities, nba_df, left_on= "NBA", right_on = "team")
    merged_df= merged_df.iloc[:, [0,1,4,6,9]]
    merged_df["W/L%"] = merged_df["W/L%"].astype("float")
    merged_df["Population (2016 est.)[8]"] = merged_df["Population (2016 est.)[8]"].astype("float")
    
    #reassigning mean ratio to NY and LS
    merged_df.loc[merged_df["Metropolitan area"] == "New York City", "W/L%"] = 0.3475
    merged_df.loc[merged_df["Metropolitan area"] == "Los Angeles", "W/L%"] = 0.4695
    merged_df = merged_df.drop_duplicates(subset= "Metropolitan area")
    merged_df = merged_df.reset_index()
    merged_df = merged_df.drop(columns = "index")
    return merged_df

def nba_correlation():
    merged_df = clean_nba_df()
    population_by_region = merged_df["Population (2016 est.)[8]"] 
    win_loss_by_region = merged_df["W/L%"]
    
    
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    
    return correlation[0]
clean_nba_df()
nba_correlation()

C:\Users\mendo\AppData\Local\Temp/ipykernel_15492/3301255494.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  cities["NBA"] = cities["NBA"].str.replace(r"\[.+\]", "")


-0.1763635064218294

# Correlation between population of city and Win/loss ratio in MLB 2018

In [2]:

mlb_df=pd.read_csv("Datasets/mlb.csv")
cities=pd.read_html("Datasets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_mlb_df():
    mlb_df=pd.read_csv("Datasets/mlb.csv")
    cities=pd.read_html("Datasets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    #cleaning mlb df
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])
    
    #cleaning cities_df
    cities["MLB"] = cities["MLB"].str.replace(r"\[.+\]", "") 
    cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue,Jays", 
                                           "CubsWhite Sox": "Cubs,White,Sox", 
                                           "DodgersAngels": "Dodgers,Angels", 
                                           "GiantsAthletics": "Giants,Athletics", 
                                           "YankeesMets": "Yankees,Mets",
                                           "Red Sox": "Red,Sox"})
    cities["MLB"] =cities["MLB"].apply(lambda x: x.split(","))
    cities = cities.explode("MLB")
    
    #merging dfs
    merged_df = pd.merge(cities, mlb_df, left_on = "MLB", right_on= "team")
    merged_df = merged_df[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W-L%"]]
    merged_df["Population (2016 est.)[8]"] = merged_df["Population (2016 est.)[8]"].astype("float")
    merged_df["W-L%"] = merged_df["W-L%"].astype("float")
    
    #reassinging mean 
    merged_df.loc[merged_df["Metropolitan area"] == "New York City", "W-L%"] = 0.546
    merged_df.loc[merged_df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.529
    merged_df.loc[merged_df["Metropolitan area"] == "San Francisco Bay Area", "W-L%"] = 0.525
    merged_df.loc[merged_df["Metropolitan area"] == "Chicago", "W-L%"] = 0.54433333
    merged_df.loc[merged_df["Metropolitan area"] == "Chicago", "W-L%"] = 0.525
    
    #Dropping indices
    merged_df = merged_df.drop_duplicates(subset= "Metropolitan area")
    merged_df = merged_df.reset_index()
    merged_df = merged_df.drop(columns = "index")
    return merged_df

def mlb_correlation(): 
    merged_df = clean_mlb_df()
    
    population_by_region = merged_df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    correlation = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return correlation [0]
clean_mlb_df()
mlb_correlation()

C:\Users\mendo\AppData\Local\Temp/ipykernel_16556/3047986418.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  cities["MLB"] = cities["MLB"].str.replace(r"\[.+\]", "")


0.16975755699575962

# Correlation between population of city and Win/loss ratio in NFL 2018

In [3]:

nfl_df=pd.read_csv("Datasets/nfl.csv")
cities=pd.read_html("Datasets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nfl_df():
    nfl_df=pd.read_csv("Datasets/nfl.csv")
    cities=pd.read_html("Datasets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    #cleaning NFL
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df["team"] = nfl_df["team"].str.replace(r"[\+]|[\*]", "")
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])
    
    #cleaning cities df
    cities["NFL"] = cities["NFL"].str.replace(r"\[.+\]", "")
    cities["NFL"] = cities["NFL"].replace({"GiantsJets":"Giants,Jets",
                                          "RamsChargers": "Rams,Chargers",
                                          "49ersRaiders": "49ers,Raiders"})
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
    cities= cities.explode("NFL")
   
    #merging df
    merged_df = pd.merge(cities, nfl_df, left_on= "NFL", right_on= "team")
    merged_df = merged_df[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL', 'W-L%', 'team']]
    
    #changing types 
    merged_df["Population (2016 est.)[8]"] = merged_df["Population (2016 est.)[8]"].astype("float")
    merged_df["W-L%"] = merged_df["W-L%"].astype("float")
    
    #Reassigning means 
    merged_df.loc[merged_df["Metropolitan area"] == "New York City", "W-L%"] = 0.2815
    merged_df.loc[merged_df["Metropolitan area"] == "Los Angeles", "W-L%"] = 0.7815
    merged_df.loc[merged_df["Metropolitan area"] == "San Francisco Bay Area" ,"W-L%"] = 0.250
    
    #dropping duplicates
    merged_df = merged_df.drop_duplicates(subset = "Metropolitan area")
    merged_df = merged_df.reset_index()
    merged_df = merged_df.drop(columns= "index")
    
    #dropping duplicates
    return merged_df


def nfl_correlation(): 
    # YOUR CODE HERE
    merged_df = clean_nfl_df()
    
    population_by_region = merged_df["Population (2016 est.)[8]"] 
    win_loss_by_region = merged_df["W-L%"]
    
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)
    return correlation[0]
    
clean_nfl_df()
nfl_correlation()

C:\Users\mendo\AppData\Local\Temp/ipykernel_16556/4214798702.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  nfl_df["team"] = nfl_df["team"].str.replace(r"[\+]|[\*]", "")
C:\Users\mendo\AppData\Local\Temp/ipykernel_16556/4214798702.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  cities["NFL"] = cities["NFL"].str.replace(r"\[.+\]", "")


0.004282141436393008

# Hypothesis test- "given that an area has two sports teams in different sports, those teams will perform the same within their respective sports"


In [163]:
mlb_df=pd.read_csv("Datasets/mlb.csv")
nhl_df=pd.read_csv("Datasets/nhl.csv")
nba_df=pd.read_csv("Datasets/nba.csv")
nfl_df=pd.read_csv("Datasets/nfl.csv")
cities=pd.read_html("Datasets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_dfs():
    mlb_df = clean_mlb_df()
    nhl_df= clean_nhl_df()
    nfl_df = clean_nfl_df()
    nba_df = clean_nba_df()

    mlb_df = mlb_df[["Metropolitan area", "W-L%"]]
    nhl_df = nhl_df[["Metropolitan area", "W-L%"]]
    nfl_df = nfl_df[["Metropolitan area", "W-L%"]]
    nba_df = nba_df[["Metropolitan area", "W/L%"]]
    nba_df = nba_df.rename(columns = {"W/L%" : "W-L%"})
    
    return (mlb_df, nhl_df, nfl_df, nba_df)

#clean_dfs()

In [162]:
def calculate_p_vals(leagues):
    p_vals = pd.DataFrame(columns=leagues.keys(), index=leagues.keys())   #creates empty df with keys of the league names passed i.e NHL, NFL etc
    for (league_name1, league_df1) in leagues.items():           #parses through name and entire df1 of first in league
        for (league_name2, league_df2) in leagues.items():       #parses through same in another df
            merged_league = pd.merge(league_df1, league_df2, on="Metropolitan area")  #merges both on area name 
            p_vals.loc[league_name1, league_name2] = stats.ttest_rel(merged_league["W-L%_x"], merged_league["W-L%_y"])[1]    #sets value of merged with results of related ttest of both dfs wi/loss ratio
            
    return p_vals

def sports_team_performance():
    # YOUR CODE HERE
#     raise NotImplementedError()
    (nfl_df, nba_df, nhl_df, mlb_df) = clean_dfs()
    
    leagues = {"NFL": nfl_df, "NBA": nba_df, "NHL": nhl_df, "MLB": mlb_df}
    p_vals_dict = calculate_p_vals(leagues)
    p_vals = pd.DataFrame(p_vals_dict).astype("float")
    return p_vals

sports_team_performance()


C:\Users\mendo\AppData\Local\Temp/ipykernel_15492/3445863027.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  nfl_df["team"] = nfl_df["team"].str.replace(r"[\+]|[\*]", "")
C:\Users\mendo\AppData\Local\Temp/ipykernel_15492/3445863027.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  cities["NFL"] = cities["NFL"].str.replace(r"\[.+\]", "")
C:\Users\mendo\AppData\Local\Temp/ipykernel_15492/3301255494.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  cities["NBA"] = cities["NBA"].str.replace(r"\[.+\]", "")
C:\Users\mendo\AppData\Local\Temp/ipykernel_15492/2550715962.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nhl_df["team"] = nhl_df["team"].str.replace("*", "")  #remov

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.835932
NBA,0.937509,NaN,0.022386,1.000000
NHL,0.030318,0.022386,NaN,0.001336
MLB,0.835932,1.000000,0.001336,NaN
